In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

from fastai.model import fit
from fastai.dataset import *

import torchtext
from torchtext import vocab, data
from torchtext.datasets import language_modeling

from fastai.rnn_reg import *
from fastai.rnn_train import *
from fastai.nlp import *
from fastai.lm_rnn import *

import dill as pickle



    Only loading the 'en' tokenizer.



In [2]:
bs,bptt = 64,70

## Language modeling

### Data

In [3]:
PATH='/home/paperspace/data/arxiv/'

df_mb = pd.read_csv(f'{PATH}arxiv.csv')
#df_all = pd.read_pickle(f'{PATH}all_arxiv.pickle')

In [4]:
def get_txt(df):
    return '<CAT> ' + df.category.str.replace(r'[\.\-]','') + ' <SUMM> ' + df.summary + ' <TITLE> ' + df.title
df_mb['txt'] = get_txt(df_mb)
#df_all['txt'] = get_txt(df_all)
df_all= df_mb
n=len(df_all); n

27188

In [4]:
os.makedirs(f'{PATH}trn/yes', exist_ok=True)
os.makedirs(f'{PATH}val/yes', exist_ok=True)
os.makedirs(f'{PATH}trn/no', exist_ok=True)
os.makedirs(f'{PATH}val/no', exist_ok=True)
os.makedirs(f'{PATH}all/trn', exist_ok=True)
os.makedirs(f'{PATH}all/val', exist_ok=True)
os.makedirs(f'{PATH}models', exist_ok=True)

In [11]:
for (i,(_,r)) in enumerate(df_all.iterrows()):
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}all/{dset}/{i}.txt', 'w').write(r['txt'])

In [12]:
for (i,(_,r)) in enumerate(df_mb.iterrows()):
    lbl = 'yes' if r.tweeted else 'no'
    dset = 'trn' if random.random()>0.1 else 'val'
    open(f'{PATH}{dset}/{lbl}/{i}.txt', 'w').write(r['txt'])

In [5]:
from spacy.symbols import ORTH

my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>', [{ORTH: '<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]



    Only loading the 'en' tokenizer.



In [ ]:
TEXT = data.Field(lower=True, tokenize=my_spacy_tok)
FILES = dict(train='trn', validation='val', test='val')
md = LanguageModelData.from_text_files(f'{PATH}all/', TEXT, **FILES, bs=bs, bptt=bptt, min_freq=10)
pickle.dump(TEXT, open(f'{PATH}models/TEXT.pkl','wb'))

In [11]:
len(md.trn_dl), md.nt, len(md.trn_ds), len(md.trn_ds[0].text)

(1094, 13463, 1, 4907202)

In [12]:
TEXT.vocab.itos[:12]

['<unk>', '<pad>', 'the', ',', '.', 'of', '-', 'and', 'a', 'to', 'in', 'we']

In [13]:
' '.join(md.trn_ds[0].text[:150])

'<cat> cssi <summ> evaluating information accuracy in social media is an increasingly important and well - studied area , but limited research has compared journalist - sourced accuracy assessments to their crowdsourced counterparts . this paper demonstrates the differences between these two populations by comparing the features used to predict accuracy assessments in two twitter data sets : credbank and pheme . while our findings are consistent with existing results on feature importance , we develop models that outperform past research . we also show limited overlap exists between the features used by journalists and crowdsourced assessors , and the resulting models poorly predict each other but produce statistically correlated results . this correlation suggests crowdsourced workers are assessing a different aspect of these stories than their journalist counterparts , but these two aspects are linked in a significant way . these differences may be explained by contrasting factual wi

### Train

In [23]:
em_sz = 200
nh = 500
nl = 3
opt_fn = partial(optim.Adam, betas=(0.7, 0.99))

In [15]:
learner = md.get_model(opt_fn, em_sz, nh, nl,
    dropout=0.05, dropouth=0.1, dropouti=0.05, dropoute=0.02, wdrop=0.2)
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
#                dropouti=0.05, dropout=0.05, wdrop=0.1, dropoute=0.02, dropouth=0.05)
learner.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
learner.clip=0.3

In [16]:
learner.fit(3e-3, 1, wds=1e-6)

A Jupyter Widget

[ 0.       4.69862  4.57728]                                  



In [17]:
learner.fit(3e-3, 3, wds=1e-6, cycle_len=1, cycle_mult=2)

A Jupyter Widget

[ 0.       4.41557  4.34256]                                  
[ 1.       4.317    4.21332]                                  
[ 2.       4.18342  4.14482]                                  
[ 3.       4.21544  4.12514]                                  
[ 4.       4.09785  4.04528]                                  
[ 5.       4.01368  3.99439]                                  
[ 6.       3.96694  3.98255]                                  



In [18]:
learner.save_encoder('adam2_enc')

In [19]:
learner.fit(3e-3, 10, wds=1e-6, cycle_len=5, cycle_save_name='adam3_10')

A Jupyter Widget

[ 0.       4.09782  4.03636]                                  
[ 1.       4.02432  3.98748]                                  
[ 2.       3.94617  3.9455 ]                                  
[ 3.       3.89137  3.91993]                                  
[ 4.       3.85898  3.91465]                                  
[ 5.       4.01525  3.97096]                                  
[ 6.       3.95356  3.93763]                                  
[ 7.       3.87995  3.90522]                                  
[ 8.       3.83657  3.88224]                                  
[ 9.       3.80372  3.87534]                                  
[ 10.        3.96715   3.93785]                               
[ 11.        3.9186    3.91091]                               
[ 12.        3.8476    3.87901]                               
[ 13.        3.81081   3.85719]                               
[ 14.        3.75302   3.85447]                               
[ 15.        3.93298   3.91325]                        

In [20]:
learner.save_encoder('adam3_10_enc')

In [21]:
learner.fit(3e-3, 8, wds=1e-6, cycle_len=10, cycle_save_name='adam3_5')

A Jupyter Widget

[ 0.       3.80733  3.8631 ]                                  
[ 1.       3.81863  3.85936]                                  
[ 2.       3.77115  3.84595]                                  
[ 3.       3.73258  3.83297]                                  
[ 4.       3.70692  3.82077]                                  
[ 5.       3.68245  3.80733]                                  
[ 6.       3.63998  3.7993 ]                                  
[ 7.       3.62947  3.79266]                                  
[ 8.       3.66046  3.78922]                                  
[ 9.       3.56934  3.79175]                                  
[ 10.        3.80811   3.85339]                               
[ 11.        3.78514   3.84789]                               
[ 12.        3.76451   3.83727]                               
[ 13.        3.75033   3.82611]                               
[ 14.        3.69643   3.81233]                               
[ 15.        3.64555   3.80331]                        

In [22]:
learner.fit(3e-3, 1, wds=1e-6, cycle_len=20, cycle_save_name='adam3_20')

A Jupyter Widget

[ 0.       3.51261  3.77196]                                  
[ 1.       3.76152  3.82596]                                  
[ 2.       3.73532  3.82539]                                  
[ 3.       3.71867  3.81948]                                  
[ 4.       3.69809  3.8182 ]                                  
[ 5.       3.69004  3.81345]                                  
[ 6.       3.72027  3.80901]                                  
[ 7.       3.64732  3.80159]                                  
[ 8.       3.6305   3.79722]                                  
[ 9.       3.60785  3.79271]                                  
[ 10.        3.59686   3.78879]                               
[ 11.        3.5682    3.78272]                               
[ 12.        3.54136   3.77841]                               
[ 13.        3.53534   3.77319]                               
[ 14.        3.54598   3.7677 ]                               
[ 15.        3.53794   3.76736]                        

In [23]:
learner.save_encoder('adam3_20_enc')

In [43]:
learner.save('adam3_20')

### Test

In [25]:
def proc_str(s): return TEXT.preprocess(TEXT.tokenize(s))
def num_str(s): return TEXT.numericalize([proc_str(s)])

In [26]:
m=learner.model

In [27]:
s="""<CAT> cscv <SUMM> algorithms that"""

In [28]:
def sample_model(m, s, l=50):
    t = num_str(s)
    m[0].bs=1
    m.eval()
    m.reset()
    res,*_ = m(t)
    print('...', end='')

    for i in range(l):
        n=res[-1].topk(2)[1]
        n = n[1] if n.data[0]==0 else n[0]
        word = TEXT.vocab.itos[n.data[0]]
        print(word, end=' ')
        if word=='<eos>': break
        res,*_ = m(n[0].unsqueeze(0))

    m[0].bs=bs

In [29]:
sample_model(m,"<CAT> csni <SUMM> algorithms that")

...use the internet of things ( iot ) to provide a reliable and reliable communication infrastructure are a key component of the future wireless networks . in this paper , we propose a novel approach to the problem of wireless sensor networks ( wsn ) . the proposed approach is 

In [30]:
sample_model(m,"<CAT> cscv <SUMM> algorithms that")

...use a single image to generate a set of images are often used to generate images with high quality . however , the quality of the generated images is not well understood . in this paper , we propose a novel method for image denoising that is based on the 

In [31]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> on ")

...the use of deep learning for image classification <eos> 

In [32]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> on ")

...the performance of wireless networks <eos> 

In [33]:
sample_model(m,"<CAT> cscv <SUMM> algorithms. <TITLE> towards ")

...a new approach for image segmentation <eos> 

In [34]:
sample_model(m,"<CAT> csni <SUMM> algorithms. <TITLE> towards ")

...a new approach to the problem of network design <eos> 

### Sentiment

In [2]:
bs,bptt = 64,70
PATH='/home/paperspace/data/arxiv/'
df_mb = pd.read_csv(f'{PATH}arxiv.csv')

from spacy.symbols import ORTH

my_tok = spacy.load('en')

my_tok.tokenizer.add_special_case('<SUMM>', [{ORTH: '<SUMM>'}])
my_tok.tokenizer.add_special_case('<CAT>', [{ORTH: '<CAT>'}])
my_tok.tokenizer.add_special_case('<TITLE>', [{ORTH: '<TITLE>'}])
my_tok.tokenizer.add_special_case('<BR />', [{ORTH: '<BR />'}])
my_tok.tokenizer.add_special_case('<BR>', [{ORTH: '<BR>'}])

def my_spacy_tok(x): return [tok.text for tok in my_tok.tokenizer(x)]

def get_txt(df):
    return '<CAT> ' + df.category.str.replace(r'[\.\-]','') + ' <SUMM> ' + df.summary + ' <TITLE> ' + df.title

df_mb['txt'] = get_txt(df_mb)



    Only loading the 'en' tokenizer.



In [20]:
df_mb['target'] = (np.random.rand(len(df_mb))*100).astype(int)

In [21]:
df_mb.loc[:2]

,link,time,favorites,rts,authors,category,published,summary,title,tweeted,txt,target
0,arxiv.org/abs/1611.10003,NaN,NaN,NaN,"[Tom A. F. Anderson, C. -H. Ruan]",q-bio.NC,2016-11-30 05:17:11,In summary of the research findings presented ...,Vocabulary and the Brain: Evidence from Neuroi...,0,<CAT> qbioNC <SUMM> In summary of the research...,57
1,arxiv.org/abs/1611.10007,NaN,NaN,NaN,"[M. Amin Rahimian, Amir G. Aghdam]",cs.SY,2016-11-30 05:37:11,"In this paper, structural controllability of a...",Structural Controllability of Multi-Agent Netw...,0,"<CAT> csSY <SUMM> In this paper, structural co...",78
2,arxiv.org/abs/1611.10010,NaN,NaN,NaN,"[Debidatta Dwibedi, Tomasz Malisiewicz, Vijay ...",cs.CV,2016-11-30 06:00:47,We present a Deep Cuboid Detector which takes ...,Deep Cuboid Detection: Beyond 2D Bounding Boxes,0,<CAT> csCV <SUMM> We present a Deep Cuboid Det...,98


In [4]:
TEXT = pickle.load(open(f'{PATH}models/TEXT.pkl','rb'))

In [7]:
class ArxivDataset(torchtext.data.Dataset):
    def __init__(self, path, text_field, label_field, **kwargs):
        fields = [('text', text_field), ('label', label_field)]
        examples = []
        for label in ['yes', 'no']:
            for fname in iglob(os.path.join(path, label, '*.txt')):
                with open(fname, 'r') as f: text = f.readline()
                # adds label and text to fields variables TEXT, ARX_LABEL    
                examples.append(data.Example.fromlist([text, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, text_field, label_field, root='.data',
               train='train', test='test', **kwargs):
        return super().splits(
            root, text_field=text_field, label_field=label_field,
            train=train, validation=None, test=test, **kwargs)

In [10]:
ARX_LABEL = data.Field(sequential=False)
splits = ArxivDataset.splits(TEXT, ARX_LABEL, PATH, train='trn', test='val')
train_ds, val_ds = splits

### Creating splits from DataFrame

In [27]:
class StructuredNLPDataset(torchtext.data.Dataset):
    def __init__(self, split_type, text_field, cat_field, target_field, dfs, **kwargs):
        fields = [("text", text_field), ("category", cat_field), ("target", target_field) ]
        examples = []
        
        df = dfs[split_type]
        dependent = 'target'
        
        for i,row in df.iterrows():
            text = row['txt']
            cat = row['category']
            label = None
            if dependent in dfs[split_type]:
                label = row[dependent]
            examples.append(data.Example.fromlist([text, cat, label], fields))
        super().__init__(examples, fields, **kwargs)

    @staticmethod
    def sort_key(ex): return len(ex.text)
    
    @classmethod
    def splits(cls, cat_field, text_field, target_field, name_as_path,
               train, val, test, dfs, **kwargs):
        return super().splits(name_as_path, 
            text_field=text_field, cat_field=cat_field, target_field=target_field, 
            train=train, validation=val, test=test, dfs=dfs, **kwargs)

In [28]:
df_train, df_val = df_mb.iloc[:25000], df_mb.iloc[25001:]
dfs = {'train': df_train, 'val': df_val, 'test': None}

In [7]:
??data.Field

In [40]:
CATEGORY = data.Field(sequential=False)
ARX_TARGET = data.Field(sequential=False, tensor_type=torch.DoubleTensor, use_vocab=False)
# path is now the name
splits = StructuredNLPDataset.splits(CATEGORY, TEXT, ARX_TARGET, '', train='train', val='val', test=None, dfs=dfs)

In [12]:
??torchtext.data.Dataset

In [31]:
splits[1].examples[0].category, splits[1].examples[0].target, splits[1].examples[0].text

('cs.SE',
 17,
 ['<cat>',
  'csse',
  '<summ>',
  'background',
  ':',
  'while',
  'bug',
  'bounty',
  'programs',
  'are',
  'not',
  'new',
  'in',
  'software',
  'development',
  ',',
  'an',
  '\n',
  'increasing',
  'number',
  'of',
  'companies',
  ',',
  'as',
  'well',
  'as',
  'open',
  'source',
  'projects',
  ',',
  'rely',
  'on',
  '\n',
  'external',
  'parties',
  'to',
  'perform',
  'the',
  'security',
  'assessment',
  'of',
  'their',
  'software',
  'for',
  '\n',
  'reward',
  '.',
  'however',
  ',',
  'there',
  'is',
  'relatively',
  'little',
  'empirical',
  'knowledge',
  'about',
  'the',
  '\n',
  'characteristics',
  'of',
  'bug',
  'bounty',
  'program',
  'contributors',
  '.',
  'aim',
  ':',
  'this',
  'paper',
  'aims',
  'to',
  '\n',
  'understand',
  'those',
  'contributors',
  'by',
  'highlighting',
  'the',
  'heterogeneity',
  'among',
  'them',
  '.',
  '\n',
  'method',
  ':',
  'we',
  'analyzed',
  'the',
  'histories',
  'of',
 

In [42]:
class TextDataLoader():
    def __init__(self, src, txt_fld, cat_fld, y_fld):
        self.src,self.txt_fld,self.cat_fld,self.y_fld = src,txt_fld,cat_fld,y_fld

    def __len__(self): return len(self.src)-1

    def __iter__(self):
        it = iter(self.src)
        for i in range(len(self)):
            b = next(it)
            yield getattr(b, self.txt_fld), getattr(b, self.cat_fld), getattr(b, self.y_fld)


class TextData(ModelData):
    def create_td(self, it): return TextDataLoader(it, self.text_fld, self.label_fld)

    @classmethod
    def from_splits(cls, path, splits, bs, text_name='text', label_name='label', cat_name='category'):
        text_fld = splits[0].fields[text_name]
        label_fld = splits[0].fields[label_name]
        cat_fld = splits[0].fields[cat_name]
        cat_fld.build_vocab(splits[0])
        # label_fld.build_vocab(splits[0])
        iters = torchtext.data.BucketIterator.splits(splits, batch_size=bs)
        trn_iter,val_iter,test_iter = iters[0],iters[1],None
        test_dl = None
        if len(iters) == 3:
            test_iter = iters[2]
            test_dl = TextDataLoader(test_iter, text_name, label_name)
        trn_dl = TextDataLoader(trn_iter, text_name, cat_name, label_name)
        val_dl = TextDataLoader(val_iter, text_name, cat_name, label_name)
        obj = cls.from_dls(path, trn_dl, val_dl, test_dl)
        obj.bs = bs
        obj.pad_idx = text_fld.vocab.stoi[text_fld.pad_token]
        obj.nt = len(text_fld.vocab)
        #obj.c = len(label_fld.vocab)
        return obj

    def get_model(self, opt_fn, max_sl, bptt, emb_sz, n_hid, n_layers, **kwargs):
        m = get_rnn_classifer(max_sl, bptt, self.bs, self.c, self.nt, emb_sz=emb_sz, n_hid=n_hid, n_layers=n_layers,
                              pad_token=self.pad_idx, **kwargs)
        model = TextModel(to_gpu(m))
        return RNN_Learner(self, model, opt_fn=opt_fn)

In [43]:
md2 = TextData.from_splits(PATH, splits, bs, label_name='target')

In [46]:
xs = next(iter(md2.trn_dl))

In [47]:
xs

(Variable containing:
     24     24     24  ...      24     24     24
    140   1838    392  ...     140   1637   1566
     26     26     26  ...      26     26     26
         ...            ⋱           ...         
    142   1444   1824  ...    1835      6     10
      1      1      1  ...      82    305    216
      1      1      1  ...   10558    142    770
 [torch.cuda.LongTensor of size 173x64 (GPU 0)], Variable containing:
    1
   21
    4
    6
    5
    6
    9
    2
   15
    5
    3
   44
   38
    8
    6
   10
    5
    2
   50
    4
    4
    4
   15
    1
   42
   31
   28
   63
   22
    2
   31
   52
   55
    2
   31
    8
    8
    1
    4
    2
   12
    4
    2
    5
    1
   20
    5
    4
    2
    2
   10
   10
   43
    1
   12
    1
    1
  111
    1
   26
    3
    1
   17
   16
 [torch.cuda.LongTensor of size 64 (GPU 0)], Variable containing:
  42
  21
  60
  48
   3
  39
  60
  25
  69
  15
  56
  46
  19
  92
  54
  55
   2
  61
  60
  84
  31
  96
  79


In [34]:
??TextDataLoader

In [14]:
??data.Field

In [39]:
#            dropout=0.3, dropouti=0.4, wdrop=0.3, dropoute=0.05, dropouth=0.2)

In [24]:
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt

def prec_at_6(preds,targs):
    precision, recall, _ = precision_recall_curve(targs==2, preds[:,2])
    print(recall[precision>=0.6][0])
    return recall[precision>=0.6][0]

In [25]:
# dropout=0.4, dropouth=0.3, dropouti=0.65, dropoute=0.1, wdrop=0.5
m3 = md2.get_model(opt_fn, 1500, bptt, emb_sz=em_sz, n_hid=nh, n_layers=nl, 
           dropout=0.1, dropouti=0.65, wdrop=0.5, dropoute=0.1, dropouth=0.3)
m3.reg_fn = partial(seq2seq_reg, alpha=2, beta=1)
m3.clip=25.

In [26]:
m3

SequentialRNN (
  (0): MultiBatchRNN (
    (encoder): Embedding(13463, 200, padding_idx=1)
    (rnns): ModuleList (
      (0): WeightDrop (
        (module): LSTM(200, 500, dropout=0.3)
      )
      (1): WeightDrop (
        (module): LSTM(500, 500, dropout=0.3)
      )
      (2): WeightDrop (
        (module): LSTM(500, 200, dropout=0.3)
      )
    )
    (dropouti): LockedDropout (
    )
    (dropouth): LockedDropout (
    )
  )
  (1): PoolingLinearClassifier (
    (decoder): Linear (600 -> 3)
    (dropout): LockedDropout (
    )
  )
)

In [27]:
m3.load_encoder(f'adam3_20_enc')
lrs=np.array([1e-4,1e-3,1e-2])

In [28]:
m3

SequentialRNN (
  (0): MultiBatchRNN (
    (encoder): Embedding(13463, 200, padding_idx=1)
    (rnns): ModuleList (
      (0): WeightDrop (
        (module): LSTM(200, 500, dropout=0.3)
      )
      (1): WeightDrop (
        (module): LSTM(500, 500, dropout=0.3)
      )
      (2): WeightDrop (
        (module): LSTM(500, 200, dropout=0.3)
      )
    )
    (dropouti): LockedDropout (
    )
    (dropouth): LockedDropout (
    )
  )
  (1): PoolingLinearClassifier (
    (decoder): Linear (600 -> 3)
    (dropout): LockedDropout (
    )
  )
)

In [45]:
m3.freeze_to(-1)
m3.fit(lrs/2, 1, metrics=[accuracy])
m3.unfreeze()
m3.fit(lrs, 1, metrics=[accuracy], cycle_len=1)

A Jupyter Widget

[ 0.       0.48448  0.38832  0.8151 ]                        



A Jupyter Widget

[ 0.       0.46593  0.42034  0.7939 ]                        



In [46]:
m3.fit(lrs, 2, metrics=[accuracy], cycle_len=4, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.43938  0.39489  0.7939 ]                        
[ 1.       0.41452  0.40475  0.78311]                        
[ 2.       0.40601  0.38734  0.79576]                        
[ 3.       0.41125  0.39618  0.79018]                        
[ 4.       0.40869  0.37752  0.80655]                        
[ 5.       0.40786  0.36441  0.81994]                        
[ 6.       0.39978  0.37931  0.80059]                        
[ 7.       0.39096  0.3716   0.81138]                        



In [47]:
prec_at_6(*m3.predict_with_targs())

0.535714285714


0.5357142857142857

In [48]:
m3.fit(lrs, 4, metrics=[accuracy], cycle_len=2, cycle_save_name='imdb2')

A Jupyter Widget

[ 0.       0.39376  0.36864  0.81808]                        
[ 1.       0.38753  0.36827  0.81808]                        
[ 2.       0.39997  0.36362  0.8192 ]                        
[ 3.       0.39627  0.36413  0.82143]                        
[ 4.       0.37498  0.36207  0.82031]                        
[ 5.       0.37599  0.36418  0.81994]                        
[ 6.       0.38193  0.36238  0.82329]                        
[ 7.       0.3761   0.36019  0.82552]                        



In [49]:
prec_at_6(*m3.predict_with_targs())

0.598639455782


0.59863945578231292